In [6]:
import pandas as pd
import re

#creating a pandas dataframe out of the csv file
traindf = pd.read_csv('../Data/08.Pubmed_PDFs_011620.csv')
testdf = pd.read_csv('../Data/04.Pubmed_PDFs_011320.csv')

out = [x[:13] for x in testdf["Identifiers"]]
#print(out)

subdf = traindf[~traindf["Identifiers"].isin(out)]
print(subdf.shape)
subdf

(88, 14)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DateTime,Keyword,Title,URL,Abstract,Description,Details,ShortDetails,Resource,Type,Identifiers,Db,EntrezUID,Properties
0,011620-185708,"(""machine learning"" OR ""deep learning"" OR ""art...",Evaluating risk prediction models for adults w...,/pubmed/31940350,BACKGROUND: The ability to predict risk allows...,"Di Tanna GL, Wirtz H, Burrows KL, Globe G.",PloS one. doi: 10.1371/journal.pone.0224135. J...,PloS one. 2020,PubMed,Journal Article,PMID:31940350,pubmed,31940350,create date: 2020/01/15 | first author: Di Tan...
1,011620-185710,"(""machine learning"" OR ""deep learning"" OR ""art...",Automated Cardiovascular Pathology Assessment ...,/pubmed/31939003,Cardiac magnetic resonance imaging provides hi...,"Lindsey T, Lee JJ.",Journal of digital imaging. doi: 10.1007/s1027...,Journal of digital imaging. 2020,PubMed,Journal Article,PMID:31939003,pubmed,31939003,create date: 2020/01/14 | first author: Lindsey T
2,011620-185718,"(""machine learning"" OR ""deep learning"" OR ""art...",Physiological Assessment of Coronary Lesions i...,/pubmed/31938934,PURPOSE OF REVIEW: Physiological assessment of...,"Chowdhury M, Osborn EA.",Current treatment options in cardiovascular me...,Current treatment options in cardiovascular me...,PubMed,"Journal Article, Review",PMID:31938934,pubmed,31938934,create date: 2020/01/15 | first author: Chowdh...
3,011620-185722,"(""machine learning"" OR ""deep learning"" OR ""art...",Of Machines and Men: Intelligent Diagnosis and...,/pubmed/31938923,"Artificial Intelligence (AI), although well es...","Cockcroft J, Avolio A.",Current hypertension reports. doi: 10.1007/s11...,Current hypertension reports. 2020,PubMed,"Journal Article, Review",PMID:31938923,pubmed,31938923,create date: 2020/01/14 | first author: Cockcr...
4,011620-185724,"(""machine learning"" OR ""deep learning"" OR ""art...",Cytosine and adenine base editing of the brain...,/pubmed/31937940,The success of base editors for the study and ...,"Levy JM, Yeh WH, Pendse N, Davis JR, Hennessey...",Nature biomedical engineering. doi: 10.1038/s4...,Nature biomedical engineering. 2020,PubMed,Journal Article,PMID:31937940,pubmed,31937940,create date: 2020/01/14 | first author: Levy JM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,011620-194624,"(""machine learning"" OR ""deep learning"" OR ""art...",Lung Innate Lymphoid Cell Composition Is Alter...,/pubmed/31394048,Comment in\n Am J Respir Crit Care Med. 202...,"Monticelli LA, Diamond JM, Saenz SA, Tait Wojn...",American journal of respiratory and critical c...,American journal of respiratory and critical c...,PubMed,Journal Article,PMID:31394048,pubmed,31394048,create date: 2020/01/01 | first author: Montic...
974,011620-195203,"(""machine learning"" OR ""deep learning"" OR ""art...",Curcumin: a therapeutic strategy in cancers by...,/pubmed/31331376,Numerous studies have presented that curcumin ...,"Vallée A, Lecarpentier Y, Vallée JN.",Journal of experimental &amp; clinical cancer ...,Journal of experimental &amp; clinical cancer ...,PubMed,"Journal Article, Review",PMID:31331376,pubmed,31331376,create date: 2019/07/22 | first author: Vallée A
976,011620-195211,"(""machine learning"" OR ""deep learning"" OR ""art...",Extra Virgin Olive Oil Contains a Phenolic Inh...,/pubmed/31331073,The lysine-specific histone demethylase 1A (LS...,"Cuyàs E, Gumuzio J, Lozano-Sánchez J, Carreras...",Nutrients. pii: E1656. doi: 10.3390/nu11071656...,Nutrients. 2019,PubMed,Journal Article,PMID:31331073,pubmed,31331073,create date: 2019/07/19 | first author: Cuyàs E
977,011620-195219,"(""machine learning"" OR ""deep learning"" OR ""art...",Effect of Vasicinone against Paraquat-Induced ...,/pubmed/31331066,Vasicinone is a quinazoline alkaloid isolated ...,"Ju DT, Sivalingam K, Kuo WW, Ho TJ, Chang RL, ...",Nutrients. pii: E1655. doi: 10.3390/nu11071655...,Nutrients. 2019,PubMed,Journal Article,PMID:31331066,pubmed,31331066,create date: 2019/07/19 | first author: Ju DT


In [159]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
import csv
vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words=text.ENGLISH_STOP_WORDSz)
documentlist = subdf['Abstract'].values
documenttitles = subdf['Title'].values
X = vectorizer.fit_transform(documentlist)
words = vectorizer.get_feature_names()

with open('../Data/TFIDFkeywords.csv', mode='w', encoding='UTF-8', newline='') as file:
    ewrite = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    ewrite.writerow(['Document', 'Word', 'TFIDFVal'])
    for doc in range(len(documentlist)) :
        wordlist = X[doc,:].nonzero()[1]
        vallist = [X[doc, x] for x in wordlist]
        topwords = sorted(range(len(vallist)), key=lambda i: vallist[i])[-10:]
        subwords = [words[i] for i in wordlist[topwords]][::-1]
        subval = [vallist[i] for i in topwords][::-1]
        for i in range(len(subval)):
            ewrite.writerow([documenttitles[doc], subwords[i], subval[i]])
print("done")
        

done


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
import csv
for gram in range(2,5):
    vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(gram,gram))
    documentlist = subdf['Abstract'].values
    documentlist = [re.sub('[0-9]+', 'XXX', document) for document in documentlist]
    documenttitles = subdf['Title'].values
    X = vectorizer.fit_transform(documentlist)
    words = vectorizer.get_feature_names()
    
    base = '../Data/TFIDF{}keywords.csv'.format(str(gram)+"gram")
    with open(base, mode='w', encoding='UTF-8', newline='') as file:
        ewrite = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        ewrite.writerow(['Document', 'Word', 'TFIDFVal'])
        for doc in range(len(documentlist)) :
            wordlist = X[doc,:].nonzero()[1]
            vallist = [X[doc, x] for x in wordlist]
            topwords = sorted(range(len(vallist)), key=lambda i: vallist[i])[-10:]
            subwords = [words[i] for i in wordlist[topwords]][::-1]
            subval = [vallist[i] for i in topwords][::-1]
            for i in range(len(subval)):
                ewrite.writerow([documenttitles[doc], subwords[i], subval[i]])
print("done")
        

done


In [13]:

[re.sub(r'\W', '', i) for i in x]

array(["BACKGROUND: The ability to predict risk allows healthcare providers to propose\nwhich patients might benefit most from certain therapies, and is relevant to\npayers' demands to justify clinical and economic value. To understand the\nrobustness of risk prediction models for heart failure (HF), we conducted a\nsystematic literature review to (1) identify HF risk-prediction models, (2)\nassess statistical approach and extent of validation, (3) identify common\nvariables, and (4) assess risk of bias (ROB).\nMETHODS: Literature databases were searched from March 2013 to May 2018 to\nidentify risk prediction models conducted in an out-of-hospital setting in adults\nwith HF. Distinct risk prediction variables were ranked according to outcomes\nassessed and incorporation into the studies. ROB was assessed using Prediction\nmodel Risk Of Bias ASsessment Tool (PROBAST).\nRESULTS: Of 4720 non-duplicated citations, 40 risk-prediction publications were\ndeemed relevant. Within the 40 public

In [21]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../Data/GoogleNews-vectors-negative300.bin.gz', binary=True)

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [28]:
#model.similarity('cardiology', 'recent findings')
model.most_similar(positive=['cardiology'])

[('interventional_cardiology', 0.7173581123352051),
 ('pediatric_cardiology', 0.7142057418823242),
 ('radiation_oncology', 0.6964948177337646),
 ('gastroenterology', 0.6928747892379761),
 ('radiology', 0.6900520324707031),
 ('cardiothoracic_surgery', 0.6833128929138184),
 ('neuroradiology', 0.6786638498306274),
 ('cardiothoracic', 0.6785849332809448),
 ('neurology', 0.6781696677207947),
 ('interventional_radiology', 0.6725773811340332)]

In [30]:
from rake_nltk import Rake #https://pypi.org/project/rake-nltk/
from nltk.corpus import stopwords 
import time
start_time = time.time()
r = Rake(stopwords = stopwords.words("english"), max_length = 5) # Uses stopwords for english from NLTK, and all puntuation characters.Please note that "hello" is not included in the list of stopwords.
counter = 0
import csv
with open('../Data/2Cardiolgydistance.csv', mode='w', encoding='UTF-8', newline='') as file:
    ewrite = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    ewrite.writerow(['Document', 'Phrase', 'RakeVal', 'SimVal'])
    for index, row in subdf.iterrows():
        rtext = re.sub('[0-9]+', 'XXX', row['Abstract'])
        a = r.extract_keywords_from_text(rtext)
        c=r.get_ranked_phrases_with_scores()
        for j in c:
            if '©' in j[1]:
                continue
            cosinevallist = []
            for k in j[1].split():
                try:
                    cval = model.similarity('cardiology', k)
                except:
                    cval = 0
                if cval != 0:
                    cosinevallist.append(cval)
            if len(cosinevallist) > 0:
                mcval = sum(cosinevallist) / len(cosinevallist)
            else:
                mcval = 0
            #print([counter, j[1], j[0], mcval])
            ewrite.writerow([row["Title"], j[1], j[0], mcval])
        if (counter % 20 == 0):
            print("--- %s seconds ---" % (time.time() - start_time))
            print(counter)
        counter+=1
print("CSV done")
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.4018361568450928 seconds ---
0
--- 0.5858025550842285 seconds ---
20
--- 0.7084746360778809 seconds ---
40
--- 0.8311467170715332 seconds ---
60
--- 0.9378607273101807 seconds ---
80
CSV done
--- 0.9707717895507812 seconds ---


In [ ]:
carddist = pd.read_csv("../Data/Cardiolgydistance.csv")